<a href="https://colab.research.google.com/github/MiaLiu123/Data-Mining/blob/main/Task1B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Task 1B

Observe that NA only happens on valence and arousal.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# 读取CSV文件
df = pd.read_csv('dataset/dataset_mood_smartphone.csv')
# df = pd.read_csv('cleaned_data.csv')


variables = df['variable'].unique()

# 为每个属性绘制包含所有患者的箱型图
for var in variables:
    plt.figure(figsize=(10, 6))

    # 创建当前属性的子集
    subset = df[df['variable'] == var]

    # 绘制箱型图，此处id作为x轴，value作为y轴
    sns.boxplot(x='id', y='value', data=subset, showfliers=True)

    # 设置图表标题和坐标轴标签
    plt.title(f'Boxplot for Variable: {var}')
    plt.xlabel('Patient ID')
    plt.ylabel('Value')

    # 显示图表
    plt.show()

### Impute NA with 0.6*day mean + 0.4(4days mean)

In [5]:
import pandas as pd
import numpy as np

# 读取CSV文件
df = pd.read_csv('dataset/dataset_mood_smartphone.csv')

# 将时间列转换为pandas的datetime类型（如果尚未转换）
df['time'] = pd.to_datetime(df['time'])

# 确保数据按时间排序
df.sort_values(by=['id', 'variable', 'time'], inplace=True)

# 函数用于计算给定行的插值
def calculate_interpolated_value(row, df):
    # 只有当值是NA的时候才填充
    if pd.isna(row['value']):
        # 当日均值计算（只取当日的数据）
        daily_values = df[(df['id'] == row['id']) & (df['variable'] == row['variable']) & (df['time'].dt.date == row['time'].date())]['value']
        daily_mean = daily_values.mean()

        # 邻近四天的相同时间段值（前后各两天，如果存在）
        date_range = pd.date_range(start=row['time'].date() - pd.Timedelta(days=2), end=row['time'].date() + pd.Timedelta(days=2), freq='D').date
        date_range = date_range[date_range != row['time'].date()]  # 去掉当日
        adjacent_values = df[(df['id'] == row['id']) & (df['variable'] == row['variable']) & (df['time'].dt.date.isin(date_range)) & (df['time'].dt.time == row['time'].time())]['value']
        adjacent_mean = adjacent_values.mean()

        # 权重计算
        if np.isnan(daily_mean) and np.isnan(adjacent_mean):
            return np.nan  # 如果两个均值都是NaN，返回NaN
        elif np.isnan(daily_mean):
            return adjacent_mean  # 如果当日均值缺失，则只取邻近四天均值
        elif np.isnan(adjacent_mean):
            return daily_mean  # 如果邻近四天均值缺失，则只取当日均值
        else:
            # 结合两个均值填充NA值（60%当日均值和40%邻近四天均值）
            return 0.6 * daily_mean + 0.4 * adjacent_mean
    else:
        return row['value']

# 应用函数计算每一行可能的插值
df['value'] = df.apply(lambda row: calculate_interpolated_value(row, df), axis=1)

# 将处理后的DataFrame保存到新的CSV文件
df.to_csv('processed_data.csv', index=False)


### Clean outliers using Quartile

In [ ]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('processed_data.csv')

# 定义一个函数，用于去除分组中的异常值
def remove_outliers(group):
    Q1 = group['value'].quantile(0.25)
    Q3 = group['value'].quantile(0.75)
    IQR = Q3 - Q1
    # 定义异常值范围
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # 只保留非异常值
    return group[(group['value'] >= lower_bound) & (group['value'] <= upper_bound)]

# 应用分组和异常值移除
cleaned_df = df.groupby(['id', 'variable']).apply(remove_outliers).reset_index(drop=True)

# 保存清洗后的数据到新的CSV文件
cleaned_df.to_csv('cleaned_data.csv', index=False)

### Calculate daily mean for each attribute of patients

In [9]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('cleaned_data.csv', parse_dates=['time'])

# 确保时间列是datetime类型
df['time'] = pd.to_datetime(df['time'])

# 提取日期部分作为一个新的列
df['date'] = df['time'].dt.date

# 按照id, variable和date分组，并计算每组的平均值
grouped_df = df.groupby(['id', 'variable', 'date'])['value'].mean().reset_index()
# 对variable为mood的值进行四舍五入
grouped_df.loc[grouped_df['variable'] == 'mood', 'value'] = grouped_df['value'].round()


# 保存到新的CSV文件
grouped_df.to_csv('daily_averages.csv', index=False)

### Transpose data be like: id, time, attr1, attr2...

In [10]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('daily_averages.csv')

# 确保时间列是datetime类型
df['date'] = pd.to_datetime(df['date'])

# 使用pivot_table来变换DataFrame
pivot_df = df.pivot_table(
    index=['id', 'date'],
    columns='variable',
    values='value',
    aggfunc='first'  # 如果同一id、同一time和variable组合有多个值，取第一个
).reset_index()

# 将列名重命名，去除多级列名中的顶层
pivot_df.columns.name = None  # 去除顶层列名
pivot_df.columns = [str(col) for col in pivot_df.columns]

# 保存到新的CSV文件
pivot_df.to_csv('transposed_data.csv', index=False)